오래전 부터 사용해오던 조금은 느슨한 프로토콜(덕 타이핑)과 비교적 근래에 도입된 ABC(Abstract Base Class)를 순서대로 다루면서 각자의 장점과 왜 ABC가 도입됐는지 그리고 어떻게 활용해야 하는지에 대해서 소개

# 1. 파이썬 문화에서의 인터페이스와 프로토콜

- 인터페이스 : 시스템에서 어떤 역할을 할 수 있게 해주는 객체의 공개 메서드의 일부
- 프로토콜 : 어떤 역할을 완수하기 위한 메서드 집합으로서의 인터페이스

# 2. 파이썬은 시퀀스를 찾아낸다

In [1]:
# __getitem__()으로 부분 구현한 시퀀스 프로토콜. 항목에 접근할 수 있고 반복할 수 있으며, in 연산자에서 사용할 수 있다.
class Foo:
    def __getitem__(self, pos):
        return range(0,30,10)[pos]

In [4]:
f = Foo()
f[1]

10

In [6]:
[i for i in f]

[0, 10, 20]

In [7]:
20 in f

True

In [8]:
15 in f

False

__iter __() 메서드가 없지만 대체 수단인 __getitem __() 메서드가 구현되어 있으므로 반복, in 연산자 등이 작동한다.

1장에서 구현한 FrenchDeck 클래스도 abc.Sequence를 상속하지 않지만, 시퀀스 프로토콜의 __getitem __()과 __len __() 메서드를 구현한다.

In [9]:
import collections

Card = collections.namedtuple('Card', ['rank','suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                      for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, pos):
        return self._cards[pos]

# 3 런타임에 프로토콜을 구현하는 멍키 패칭

In [12]:
#FrenchDeck 클래스에 shuffle() 메서드를 구현할 필요가 없는 것은 random.shuffle() 함수가 
#시퀀스 객체 안의 항목들을 섞어주기 때문이다. 그러나 다음과 같이 입력하면 예외가 발생하는데, 이는 FrenchDeck 
#객체가 할당을 지원하지 않기 때문이다.

from random import shuffle

deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

In [14]:
def set_card(deck, pos, card):
    deck._cards[pos] = card

FrenchDeck.__setitem__ = set_card

In [36]:
# 멍키 패칭 : 소스 코드를 건드리지 않고 런타임에 클래스나 모듈을 변경하는 행위
shuffle(deck)
deck[:5]

[Card(rank='K', suit='clubs'),
 Card(rank='A', suit='spades'),
 Card(rank='8', suit='spades'),
 Card(rank='9', suit='spades'),
 Card(rank='9', suit='clubs')]

# 4. 알렉스 마르텔리의 물새

ABC는 '시퀀스'나 '정확한 숫자' 같은 일종의 프레임워크가 소개하는 상당히 광범위한 개념이나 추상성을 담기 위한 것이다. 이 책을 읽는 독자라면 ABC를 새로 만들 필요가 전혀 없을 것이다. 단지 기존 ABC를 제대로 사용하는 것만으로도 잘못된 설계 위험 없이 99.9%의 혜택을 볼 것이다.

# 5 ABC 상속하기

In [39]:
# FrenchDeck2를 collections.MutableSequence의 서브클래스로 선언

import collections

Card = collections.namedtuple('Card', ['rank','suit'])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2,11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                      for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]
    
    def __setitem__(self, position, value):
        self._cards[position] = value
        
    # MutableSequence를 상속하므로 이 클래스의 추상 메서드인 
    # __delitem__() 도 구현해야함
    def __delitem__(self, position):
        del self._cards[position]
        
    # insert() 또한 추상 메서드
#     def insert(self, position, value):
#         self._cards.insert(position, value)

In [38]:
frenchdeck2 = FrenchDeck2()

TypeError: Can't instantiate abstract class FrenchDeck2 with abstract method insert

TypeError : __delitem __(), __insert __()를 가진 추상 클래스 FrenchDeck2의 객체를 생성할 수 없습니다.

사용하지도 않는 __delitem __(), __insert __()메서드를 구현해야한다.  
MutableSequence ABC가 요구하는 사항이기 때문이다.

# 6. 표준 라이브러리 ABC  

대부분의 ABC는 collections.abc 모듈에 정의되어 있다. 어떤 ABC들이 있는지 살펴보자.

<img src="https://camo.githubusercontent.com/f4801bf49fe492d95e350932577c13f14ac8f8df036720082f970689cb8377aa/68747470733a2f2f757365722d696d616765732e67697468756275736572636f6e74656e742e636f6d2f33333839313136342f33393336333932382d65396261343532302d343965302d313165382d396363352d3237333133346531366466342e706e67">


### 6.2 ABC의 숫자탑 

numbers 패키지는 다음과 같이 계층 구조로 이루어져 있다. Number가 최상위 슈퍼클래스이며, Integral까지 내려간다.

- Number
- Complex 
- Real
- Rational
 - Integral

In [47]:
#정수형인지 검사해야 하는 경우 isinstance(x, numbers.Integral)을 이용하면 된다.

import numbers

a = 5
b = 3
a*b, isinstance(a, numbers.Integral)

(15, True)

# 7.ABC의 정의와 사용
  
  다음 상황 가정  
  
      웹사이트나 모바일 앰에서 광고를 무작위 순으로 보여주어야 하지만, 광고 목록에 들어 있는 광고를 모두 보여주기 전까지는 같은 광고를 반복하면 안된다.

In [48]:
# 추상 메서드 두 개와 구상 메서드 두개를 가진 Tombola ABC

import abc

class Tombola(abc.ABC): # ABC 정의하기 위해 상속
    
    @abc.abstractmethod
    def load(self, iterable):
        """iterable의 항목들을 추가한다."""
        
    @abc.abstractmethod
    def pick(self):
        """무작위로 항목을 하나 제거하고 반환한다.
        객체가 비어 있을 때 이 메서드를 실행하면 'LookupError'가 발생한다.
        """
        
    def loaded(self): # ABC 에도 구상 메서드가 들어갈 수 있다
        """최소 한 개의 항목이 있으면 True, 아님 False 반환"""
        # ABC의 구상 메서드는 반드시 ABC에 정의된 인터페이스, 즉
        # ABC의 다른 구상 메서드나 추상 메서드, 혹은 프로퍼티만 사용해야 한다.
        return bool(self.inspect())
    
    def inspect(self):
        """현재 안에 있는 항목들로 구성된 정렬된 튜플 반환"""
        items = []
        while True:
            try:
                # pick() 을 계속 호출해서 Tombola 객체를 비움
                items.append(self.pick())
            except LookupError:
                break
        self.load(items) # load 메서드를 호출해서 다시 넣는다
        return tuple(sorted(items))

load() : 항목을 컨테이너 안에 넣는다.

pick() : 컨테이너 안에서 무작위로 항목 하나를 꺼내서 반환한다.

loaded() : 컨테이너 안에 항목이 하나 이상 들어 있으면 True 반환한다.

inspect() : 내용물을 변경하지 ㅇ낳고 현재 컨테이너 안에 들어있는 항목을 튜플로 만들어 반환한다.

inspect() 메서드는 멍청해보이지만, 이 코드의 핵심은 ABC 안에서 인터페이스에 정의된 다른 메서드만 이용하는 한 ABC에 구상 메서드를 제공하는 것도 가능하다는 점을 보여주는 것이다. 물론 상속하면서 오버라이드 해도 된다.

loaded() 연산자는 상당히 값비싼 연산을 수행하므로 구상 서브클래스에서 더 잘 구현하는 것이 좋다.

LookupError를 선택한 이유는 Tombola의 서브클래스의 데이터 구조체에서 발생할 가능성이 높은 IndexError와 KeyError 에러의 상위 클래스이기 때문이다.

IndexError : 시퀀스에서 마지막 이후 인덱스를 호출 시 발생
KeyError : 매핑에 존재하지 않는 키로 호출 시 발생
실제로 ABC가 인터페이스 검사를 제대로 수행하는지 확인해보자.

In [50]:
class Fake(Tombola):
    def pick(self):
        return 13

In [54]:
Fake

__main__.Fake

In [56]:
#load()를 구현하지 않았으므로 객체 생성 시 TypeError 발생 ()
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract method load